# Secret Sharing



## Shamir Schemes



### Shamir Secret Sharing Example



In [3]:
# Define the field
F = GF(211)

# Define the Vandermonde matrix A
A = Matrix(F, [[1, 1, 1], [1, 2, 4], [1, 3, 9]])

# Define the output vector b
b = vector(F, [4, 5, 6])

# Solve the linear equation Ax = b
x = A.solve_right(b)

# Extract the coefficients
s0, s1, s2 = x

# The secret is s0
print(f"The secret is {s0}")

# The polynomial is s(x) = s0 + s1*x + s2*x^2
print(f"The polynomial is s(x) = {s0} + {s1}*x + {s2}*x^2")


The secret is 3
The polynomial is s(x) = 3 + 1*x + 0*x^2


### Shamir Threshold Scheme \(4,31\) Example



In [4]:
# Import the matrix module from SageMath
from sage.matrix.constructor import Matrix

# Define the prime number and the known shares
p = 223
shares = [(1, 8), (2, 16), (3, 32), (4, 64)]

# Construct matrix A and vector b
A = []
b = []

for x, y in shares:
    A.append([x**i for i in range(len(shares))])  # Modify the range to match the length of shares
    b.append(y)

A = Matrix(A)%p  # Apply modulo p to the matrix A
print(f"Matrix A is\n{A}")
b = Matrix(b).transpose()%p  # Apply modulo p to the vector b
print(f"Matrix b is\n{b}")

# Calculate the adjoint (adjugate) of A and the determinant modulo p
A_adj = A.adjugate()%p
print(f"The Adj of A is\n{A_adj}")
det_A = A.determinant()%p
print(f"det A is {det_A}")

# Calculate det_A inverse mod p
det_A_inv = power_mod(det_A, -1, p)

# Calculate the coefficients using the formula: coef = (adj(A) * b * det(A)^(-1)) mod p
coefficients = (A_adj * b * det_A_inv) % p

# Extract values for better readability
s0, s1, s2, s3 = coefficients[0,0], coefficients[1,0], coefficients[2,0], coefficients[3,0]

# Print the polynomial and the secret
print(f"The polynomial p(x) = {s3}x^3 + {s2}x^2 + {s1}x + {s0}")
print(f"The secret is {s0}")


Matrix A is
[ 1  1  1  1]
[ 1  2  4  8]
[ 1  3  9 27]
[ 1  4 16 64]
Matrix b is
[ 8]
[16]
[32]
[64]
The Adj of A is
[ 48 151  48 211]
[171 114 139  22]
[ 18 175  42 211]
[221   6 217   2]
det A is 12
The polynomial p(x) = 150x^3 + 219x^2 + 85x + 0
The secret is 0


### Shamir Threshold Scheme \(3,n\) Example With Missing Share



In [5]:
# Version is (3, 3)
# Import the matrix module from SageMath
from sage.matrix.constructor import Matrix

# Define the prime number and the known shares
p = 191
shares = [(18, 111), (7, 159), (6, 64)]

# Construct matrix A and vector b
A = []
b = []

for x, y in shares:
    A.append([x**i for i in range(3)])
    b.append(y)

A = Matrix(A)%p
print(f"Matrix A is\n{A}")
b = Matrix(b).transpose()
print(f"Matrix b is\n{b}")

# Calculate the adjoint (adjugate) of A and the determinant
A_adj = A.adjugate()%p
print(f"The Adj of A is\n{A_adj}")
det_A = A.determinant()%p
det_A_inv = inverse_mod(det_A, p)
print(f"det A is {det_A_inv}")
A_inv = (A_adj * det_A_inv)%p
print(f"The Inverse of A is\n{A_inv}")
sb = (A_inv*b)%p
print(f"The final matrix vector is\n{sb}")


# Calculate det_A inverse mod p
det_A_inv = power_mod(det_A, -1, p)

# Calculate the coefficients using the formula: coef = (adj(A) * b * det(A)^(-1)) mod p
coefficients = (A_adj * b * det_A_inv) % p

# Extract values for better readability
s0, s1, s2 = coefficients[0,0], coefficients[1,0], coefficients[2,0]

# Print the polynomial and the secret
print(f"The polynomial p(x) = {s2}x^2 + {s1}x + {s0}")
print(f"The secret is {s0}")

# To find the missing share (x, *)
x = 5
missing_share = (s0 + s1*x + s2*x**2) % p
print(f"The missing share ({x}, *) is ({x}, {missing_share})")


Matrix A is
[  1  18 133]
[  1   7  49]
[  1   6  36]
Matrix b is
[111]
[159]
[ 64]
The Adj of A is
[149 150 142]
[ 13  94  84]
[190  12 180]
det A is 68
The Inverse of A is
[  9  77 106]
[120  89 173]
[123  52  16]
The final matrix vector is
[162]
[152]
[ 25]
The polynomial p(x) = 25x^2 + 152x + 162
The secret is 162
The missing share (5, *) is (5, 19)


### Shamir Secret Sharing Setup



In [6]:
import random as py_random
from sage.all import *

def generate_shamir_scheme(t, w, m):
    # Generate a prime p larger than both w and m
    p = next_prime(max(w, m))
    
    # Coefficients for the polynomial, with s_0 = m
    coeffs = [m]
    
    # Randomly select t-1 coefficients from Z_p
    for i in range(1, t):
        coeffs.append(py_random.randint(1, p-1))
    
    # Create the polynomial s(x) = m + s_1 * x + ... + s_{t-1} * x^{t-1}
    R = PolynomialRing(GF(p), 'x')
    s_x = R(coeffs)
    
    # Display the generated prime and polynomial
    print(f"Prime p: {p}")
    print(f"Polynomial s(x): {s_x}")
    
    return p, s_x

# Example usage
t = 3
w = 8
m = 1234
generate_shamir_scheme(t, w, m)


Prime p: 1237
Polynomial s(x): 237*x^2 + 755*x + 1234


(1237, 237*x^2 + 755*x + 1234)

### Shamir Polynomial Recovery



In [7]:
def recover_polynomial(t, shares, p):
    # Define the base ring and polynomial ring
    R = PolynomialRing(GF(p), 'x')
    x = R.gen()
    
    # Initialize zero polynomial
    s_x = 0
    
    # Lagrange interpolation
    for j in range(t):
        x_j, y_j = shares[j]
        l_j = 1
        for m in range(t):
            x_m, _ = shares[m]
            if m == j:
                continue
            l_j *= (x - x_m) / (x_j - x_m)
        s_x += y_j * l_j
    
    # Display recovered polynomial
    print(f"Recovered polynomial s(x) modulo {p}: {s_x}")
    
    return s_x

# Example usage
p = 227  # Example prime number
t = 3  # Threshold
shares = [(1, 4), (2, 8), (3, 16)]  # Example shares

recover_polynomial(t, shares, p)


Recovered polynomial s(x) modulo 227: 2*x^2 + 225*x + 4


2*x^2 + 225*x + 4

## Alternate Schemes and Shamir Lagrange Extensions



### 3D Implementation of Blakley's Geometric Secret Sharing Scheme



In [8]:
import random

# Initialize variables
p = 167  # prime number
m = 45  # message
w = 5  # number of shares

# Create a point Q = (45, x2, x3) in R^3
x2 = 11
x3 = 153
Q = (m, x2, x3)

# Function to create a hyperplane equation passing through Q
def create_hyperplane(Q, a, b):
    (m, x2, x3) = Q
    c = (a * m + b * x2 + x3) % p
    return (a, b, c)

# Generate w hyperplanes
hyperplanes = []
for i in range(w):
    a = random.randint(1, p - 1)
    b = random.randint(1, p - 1)
    hyperplanes.append(create_hyperplane(Q, a, b))

# Output the point Q and hyperplanes
print("Point Q:", Q)
print("Hyperplanes:")
for h in hyperplanes:
    print(f"z = {h[0]}x + {h[1]}y + {h[2]}")


Point Q: (45, 11, 153)
Hyperplanes:
z = 62x + 22y + 12
z = 98x + 92y + 64
z = 149x + 51y + 71
z = 35x + 73y + 26
z = 10x + 9y + 34


### Matrix Formulation and Inversion in 3D Blakley's Secret Sharing Scheme



In [9]:
# Initialise variables
p = 167  # Prime number for modulo operation
m = 45  # Message (secret)
t = 3  # Threshold
n = 5  # Number of hyperplanes

# Manually specify x2 and x3 for point Q
x2 = 11
x3 = 153

# Initialize point Q = (m, x2, x3)
Q = (m, x2, x3)

# Generate random coefficients for hyperplanes
random.seed()
hyperplanes = []
for i in range(n):
    a = random.randint(1, p - 1)
    b = random.randint(1, p - 1)
    c = (a * m + b * x2 + x3) % p
    hyperplanes.append((a, b, c))

# Print generated hyperplanes
print(f"Point Q: {Q}")
print("Hyperplanes:")
for a, b, c in hyperplanes:
    print(f"z = {a}x + {b}y + {c}")

# Generate and print matrices Mi and constant vectors Ct for each group of t hyperplanes
for i in range(0, len(hyperplanes) - t + 1, t):
    M_i = Matrix(t, 3)
    C_t = Matrix(t, 1)
    for j in range(t):
        a, b, c = hyperplanes[i + j]
        M_i[j] = [a, b, -1]
        C_t[j, 0] = -c % p
    print(f"Matrix M_{int(i/t) + 1}:")
    print(M_i)
    print(f"Constant vector C_{int(i/t) + 1}: \n{C_t}")
    M_i_det = M_i.det()%p
    print(f"Determinant of M_{int(i/t) + 1}: {(M_i.det())%p}")
    M_adj = M_i.adjugate()%p
    print(f"The Adj of M is\n{M_adj}")
    inv_M_i = (M_adj*M_i_det)%p
    print(f"The inverse of Matrix M{int(i/t) + 1}:")
    print(inv_M_i)


Point Q: (45, 11, 153)
Hyperplanes:
z = 115x + 21y + 48
z = 104x + 108y + 9
z = 138x + 37y + 90
z = 48x + 129y + 58
z = 73x + 90y + 86
Matrix M_1:
[115  21  -1]
[104 108  -1]
[138  37  -1]
Constant vector C_1: 
[119]
[158]
[ 77]
Determinant of M_1: 6
The Adj of M is
[ 96 151  87]
[133  23  11]
[133 146  49]
The inverse of Matrix M1:
[ 75  71  21]
[130 138  66]
[130  41 127]


### Generating Invertible Sets of Hyperplanes in 3D Blakley's Scheme



In [10]:
# Initialize parameters
p = 167  # Prime number for modulo operation
m = 45  # Message (secret)
w = 5  # Number of hyperplanes

# Manually specify x2 and x3 for point Q
x2 = 11
x3 = 153

# Initialize point Q = (m, x2, x3)
Q = (m, x2, x3)

# Initialize list to hold hyperplanes
hyperplanes = []

# Function to calculate the determinant modulo p for a matrix constructed from three planes
def mod_det(plane1, plane2, plane3):
    M = Matrix(3, 3)
    M[0] = [plane1[0], plane1[1], -1]
    M[1] = [plane2[0], plane2[1], -1]
    M[2] = [plane3[0], plane3[1], -1]
    return M.det() % p

# Step 1: Generate first two planes that are not parallel and contain Q
while len(hyperplanes) < 2:
    a = randint(1, p - 1)
    b = randint(1, p - 1)
    c = (a * m + b * x2 + x3) % p
    if len(hyperplanes) == 0 or (a, b) != (hyperplanes[0][0], hyperplanes[0][1]):
        hyperplanes.append((a, b, c))

# Step 2: Generate additional planes, ensuring that they form an invertible matrix with every pair of existing planes
while len(hyperplanes) < w:
    a = randint(1, p - 1)
    b = randint(1, p - 1)
    c = (a * m + b * x2 + x3) % p
    valid = True
    for i in range(len(hyperplanes)):
        for j in range(i+1, len(hyperplanes)):
            if gcd(mod_det(hyperplanes[i], hyperplanes[j], (a, b, c)), p) != 1:
                valid = False
                break
        if not valid:
            break
    if valid:
        hyperplanes.append((a, b, c))

# Print the results
print(f"Point Q: {Q}")
print("Hyperplanes:")
for a, b, c in hyperplanes:
    print(f"z = {a}x + {b}y + {c}")


Point Q: (45, 11, 153)
Hyperplanes:
z = 139x + 71y + 8
z = 153x + 82y + 91
z = 62x + 127y + 165
z = 115x + 126y + 34
z = 61x + 77y + 71


### Lagrange Interpolation for Polynomial Reconstruction in Shamir's Scheme



In [11]:
R.<x> = PolynomialRing(GF(131))
shares = [(6, 12), (10, 92), (8, 45)]
t = len(shares)  # Number of shares, which is 3 in this case

# Initialize list to hold Lagrange basis polynomials
lagrange_basis_polys = []

# Calculate Lagrange basis polynomials
for j in range(t):
    lj = R(1)  # Initialize lj as 1
    xj, _ = shares[j]
    for m in range(t):
        if m != j:
            xm, _ = shares[m]
            lj *= (x - xm) / (xj - xm)
    lagrange_basis_polys.append(lj)

# Calculate the secret polynomial
secret_poly = R(0)  # Initialize the secret polynomial as 0
for j in range(t):
    _, yj = shares[j]
    secret_poly += yj * lagrange_basis_polys[j]

# Print Lagrange basis polynomials
for i, poly in enumerate(lagrange_basis_polys):
    print(f"l_{i}(x) = {poly}")

print(f"The secret polynomial is f(x) = {secret_poly}")

# Extract and print the constant term, which is the secret
print(f"The secret is {secret_poly.constant_coefficient()}")

# To find the missing share (5, *)
x = 2
missing_share = (s0 + s1*x + s2*x**2) % p
print(f"The missing share ({x}, *) is ({x}, {missing_share})")

l_0(x) = 82*x^2 + 96*x + 10
l_1(x) = 82*x^2 + 31*x + 6
l_2(x) = 98*x^2 + 4*x + 116
The secret polynomial is f(x) = 100*x^2 + 123*x + 128
The secret is 128
The missing share (2, *) is (2, 65)


### Hyperplane Intersection for Secret Reconstruction in 3D Space for Blakley's Secret Sharing Scheme



In [12]:
# Import SageMath modules
from sage.matrix.constructor import Matrix

# Initialize parameters
p = 191  # Prime number for modulo operation
n = 10  # Number of hyperplanes
k = 3  # Threshold

# Given keys for Alice, Bob, and Charles
keys = [((109, 172), 54), ((44, 174), 42), ((156, 112), 80)]

# Initialize matrix and constant vector for solving the system
M = Matrix(k, 3)
C = Matrix(k, 1)


# Populate M and C from given keys
for i, ((a, b), c) in enumerate(keys):
    M[i] = [a, b, -1]
    C[i, 0] = -c % p

print(M)
print(C)

# Calculate the determinant mod p
M_det = M.det() % p

# Check that the determinant is not zero and is relatively prime to p
if M_det == 0:
    print("Determinant is zero. The matrix is singular.")
elif gcd(M_det, p) != 1:
    print(f"Determinant is not relatively prime to {p}. Cannot proceed.")
else:
    # Calculate adjoint (adjugate) matrix mod p
    M_adj = M.adjugate() % p

    # Calculate inverse of determinant mod p
    M_det_inv = inverse_mod(M_det, p)

    # Compute the inverse of the matrix M
    M_inv = (M_adj * M_det_inv) % p

    # Compute the point Q
    Q = (M_inv * C) % p

    # Output
    print(f"The determinant of M: {M_det}")
    print(f"The adjoint of M:\n{M_adj}")
    print(f"The inverse of M:\n{M_inv}")
    print(f"The point Q: {Q}")
    print(f"The secret x0 is {Q[0,0]}")


[109 172  -1]
[ 44 174  -1]
[156 112  -1]
[137]
[149]
[111]
The determinant of M: 14
The adjoint of M:
[129  60   2]
[ 79  47  65]
[131 108 129]
The inverse of M:
[132 168  82]
[183  17 182]
[ 23  35 132]
The point Q: [75]
[56]
[98]
The secret x0 is 75
